<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES
================================================================================
Trabajo Fin de Master: Evaluacion Comparativa de Tecnicas de Segmentacion
en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universidad Oberta de Cataluna (UOC)
Fecha: Diciembre 2025

DESCRIPCION:
Fase 2B del analisis comparativo de modelos de segmentacion. Fusiona las metricas
de segmentacion calculadas en Fase 2A con las caracteristicas fotograficas
extraidas en Fase 1, permitiendo analizar como los parametros de captura
(apertura, ISO, exposicion) y las propiedades de imagen (contraste, nitidez,
complejidad textural) afectan al rendimiento de cada modelo.

DEPENDENCIAS DE DATOS:
- Requiere Fase 2A completada (CSVs de metricas por modelo)
- Requiere JSONs de caracteristicas fotograficas (20 archivos)
- Genera dataset fusionado para fases posteriores

ANALISIS INCLUIDOS:
1. Consolidacion de CSVs de metricas de todos los modelos
2. Extraccion de caracteristicas fotograficas clave (20 variables)
3. Fusion de datasets por codigo de fotografia
4. Correlaciones globales: caracteristicas vs IoU
5. Correlaciones por modelo: sensibilidad diferencial
6. Test de hipotesis especificas (bokeh, contraste, exposicion, fondo)
7. Tamano de efecto entre modelos (Cohen's d, eta cuadrado)
8. Estadisticas descriptivas y resumen ejecutivo

CARACTERISTICAS FOTOGRAFICAS EXTRAIDAS:
- EXIF: apertura, ISO, exposicion, focal
- Calidad: brillo, subexposicion, sobreexposicion, rango dinamico
- Contraste: RMS, nitidez Laplacian, SNR
- Textura: homogeneidad GLCM, entropia Haralick, contraste Haralick
- Frecuencia: ratio alta frecuencia
- Saliencia: centro, ratio centro/periferia, distancia centroide
- Color: entropia, saturacion

ESTRUCTURA DE SALIDA:
/TFM/3_Analisis/fase2b_correlaciones/
├── metricas_fusionadas.csv           # Dataset completo fusionado
├── caracteristicas_fotograficas.csv  # Caracteristicas por foto
├── correlaciones_globales.csv        # Correlaciones caracteristicas vs IoU
├── correlaciones_por_modelo.csv      # Correlaciones segmentadas por modelo
├── test_hipotesis.json               # Resultados de tests estadisticos
├── tamano_efecto.json                # Cohen's d entre modelos

================================================================================
"""

"\n================================================================================\nFASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES\n================================================================================\nTrabajo Fin de Master: Evaluacion Comparativa de Tecnicas de Segmentacion\nen Fotografia de Retrato\n\nAutor: Jesus L.\nUniversidad: Universidad Oberta de Cataluna (UOC)\nFecha: Diciembre 2025\n\nDESCRIPCION:\nFase 2B del analisis comparativo de modelos de segmentacion. Fusiona las metricas\nde segmentacion calculadas en Fase 2A con las caracteristicas fotograficas\nextraidas en Fase 1, permitiendo analizar como los parametros de captura\n(apertura, ISO, exposicion) y las propiedades de imagen (contraste, nitidez,\ncomplejidad textural) afectan al rendimiento de cada modelo.\n\nDEPENDENCIAS DE DATOS:\n- Requiere Fase 2A completada (CSVs de metricas por modelo)\n- Requiere JSONs de caracteristicas fotograficas (20 archivos)\n- Genera dataset fusionado para fases poster

In [5]:
# =============================================================================
# IMPORTS
# =============================================================================

import json
import logging
import sys
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from scipy import stats


In [6]:
# =============================================================================
# FUNCIONES DE UTILIDAD PARA SERIALIZACION
# =============================================================================

def convertir_a_serializable(obj: Any) -> Any:
    """
    Convierte tipos numpy y pandas a tipos Python nativos para JSON.

    Args:
        obj: Objeto a convertir

    Returns:
        Objeto con tipos serializables
    """
    if isinstance(obj, dict):
        return {k: convertir_a_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convertir_a_serializable(v) for v in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    return obj

In [7]:
# =============================================================================
# CONFIGURACION
# =============================================================================

@dataclass
class ConfiguracionFase2B:
    """
    Configuracion central para el pipeline de Fase 2B.

    Attributes:
        ruta_base_tfm: Ruta raiz del proyecto TFM
        ruta_csvs_fase2a: Directorio con CSVs de metricas de Fase 2A
        ruta_jsons_caracteristicas: Directorio con JSONs de caracteristicas
        ruta_salida: Directorio para resultados de Fase 2B
        nivel_significancia: Alpha para tests estadisticos (default 0.05)
        min_observaciones_correlacion: Minimo de observaciones para calcular correlacion
    """
    ruta_base_tfm: Path
    ruta_csvs_fase2a: Path = None
    ruta_jsons_caracteristicas: Path = None
    ruta_salida: Path = None
    nivel_significancia: float = 0.05
    min_observaciones_correlacion: int = 10

    def __post_init__(self):
        """Inicializa rutas derivadas si no se especifican."""
        if self.ruta_csvs_fase2a is None:
            self.ruta_csvs_fase2a = self.ruta_base_tfm / "3_Analisis" / "fase2_evaluacion" / "metricas_agregadas"
        if self.ruta_jsons_caracteristicas is None:
            self.ruta_jsons_caracteristicas = self.ruta_base_tfm / "0_Imagenes" / "caracteristicas"
        if self.ruta_salida is None:
            self.ruta_salida = self.ruta_base_tfm / "3_Analisis" / "fase2b_correlaciones"

In [8]:
# =============================================================================
# CONSTANTES: CARACTERISTICAS FOTOGRAFICAS A EXTRAER
# =============================================================================

CARACTERISTICAS_CLAVE = {
    # EXIF - Parametros de captura
    'foto_apertura': ('metadatos_exif', 'apertura_fnumber'),
    'foto_iso': ('metadatos_exif', 'iso'),
    'foto_exposicion': ('metadatos_exif', 'tiempo_exposicion_segundos'),
    'foto_focal': ('metadatos_exif', 'distancia_focal'),

    # Calidad - Exposicion
    'foto_brillo': ('calidad', 'exposicion', 'brillo_medio'),
    'foto_subexp_pct': ('calidad', 'exposicion', 'sub_expuesto_pct'),
    'foto_sobreexp_pct': ('calidad', 'exposicion', 'sobre_expuesto_pct'),
    'foto_rango_dinamico': ('calidad', 'exposicion', 'rango_dinamico'),

    # Calidad - Contraste y Nitidez
    'foto_contraste_rms': ('calidad', 'contraste', 'rms'),
    'foto_nitidez_laplacian': ('calidad', 'nitidez', 'laplacian'),
    'foto_snr': ('calidad', 'ruido', 'snr'),

    # Texturas
    'foto_homogeneity': ('texturas', 'glcm', 'homogeneity_mean'),
    'foto_haralick_entropy': ('texturas', 'haralick', 'entropy'),
    'foto_haralick_contrast': ('texturas', 'haralick', 'contrast'),

    # Frecuencia
    'foto_freq_ratio_alta': ('analisis_frecuencia', 'ratio_alta'),

    # Saliencia
    'foto_saliencia_centro': ('saliencia_visual', 'distribucion_espacial', 'saliencia_centro'),
    'foto_ratio_centro_periferia': ('saliencia_visual', 'distribucion_espacial', 'ratio_centro_periferia'),
    'foto_centroide_distancia': ('saliencia_visual', 'centroide', 'distancia_desde_centro'),

    # Color
    'foto_entropia_color': ('estadisticas_color', 'global', 'entropia'),
    'foto_saturacion': ('estadisticas_color', 'hsv', 'saturation_mean'),
}

In [9]:
# =============================================================================
# CLASE: CARGADOR DE DATOS
# =============================================================================

class CargadorDatosFase2B:
    """
    Gestiona la carga y consolidacion de datos para Fase 2B.

    Responsabilidades:
    - Cargar y consolidar CSVs de metricas de todos los modelos
    - Cargar y extraer caracteristicas de JSONs fotograficos
    - Fusionar ambos datasets
    """

    def __init__(self, config: ConfiguracionFase2B, logger: logging.Logger):
        """
        Inicializa el cargador de datos.

        Args:
            config: Configuracion de Fase 2B
            logger: Logger para registro de eventos
        """
        self.config = config
        self.logger = logger

    def cargar_csvs_metricas(self, rutas_csv: Dict[str, Path]) -> pd.DataFrame:
        """
        Consolida multiples CSVs de metricas en un unico DataFrame.

        Args:
            rutas_csv: Diccionario {nombre_modelo: ruta_csv}

        Returns:
            DataFrame consolidado con todas las metricas
        """
        self.logger.info("Cargando CSVs de metricas de Fase 2A...")

        dataframes = []

        for modelo, ruta in rutas_csv.items():
            try:
                df = pd.read_csv(ruta)

                # Corregir nombre de modelo si es necesario
                if modelo == 'sam2_prompts' and 'modelo' in df.columns:
                    df['modelo'] = 'sam2_prompts'

                self.logger.info(f"  {modelo}: {len(df)} filas, {len(df.columns)} columnas")
                dataframes.append(df)

            except Exception as e:
                self.logger.error(f"  Error cargando {modelo}: {e}")

        if not dataframes:
            raise ValueError("No se pudieron cargar CSVs de metricas")

        # Concatenar todos los DataFrames
        df_consolidado = pd.concat(dataframes, ignore_index=True)

        self.logger.info(f"Total consolidado: {len(df_consolidado)} filas")

        return df_consolidado

    def extraer_caracteristicas_json(self, ruta_json: Path) -> Dict[str, Any]:
        """
        Extrae caracteristicas clave de un JSON de fotografia.

        Args:
            ruta_json: Ruta al archivo JSON

        Returns:
            Diccionario con caracteristicas extraidas
        """
        with open(ruta_json, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Obtener nombre de foto
        nombre = data.get('metadatos_archivo', {}).get('nombre_archivo', '')
        nombre = nombre.replace('.jpg', '').replace('.JPG', '')

        caracteristicas = {'codigo_foto': nombre}

        # Extraer cada caracteristica definida
        for nombre_col, ruta_keys in CARACTERISTICAS_CLAVE.items():
            valor = self._extraer_valor_anidado(data, ruta_keys)

            # Convertir a numerico si es posible
            if valor is not None and not isinstance(valor, (int, float)):
                try:
                    valor = float(valor)
                except (ValueError, TypeError):
                    valor = None

            caracteristicas[nombre_col] = valor

        return caracteristicas

    def _extraer_valor_anidado(self, data: Dict, keys: Tuple) -> Any:
        """
        Extrae un valor de un diccionario anidado siguiendo una ruta de claves.

        Args:
            data: Diccionario fuente
            keys: Tupla de claves para navegar

        Returns:
            Valor extraido o None si no existe
        """
        resultado = data
        for key in keys:
            if isinstance(resultado, dict) and key in resultado:
                resultado = resultado[key]
            else:
                return None
        return resultado

    def cargar_caracteristicas_fotograficas(self, rutas_json: List[Path]) -> pd.DataFrame:
        """
        Carga caracteristicas de multiples JSONs fotograficos.

        Args:
            rutas_json: Lista de rutas a archivos JSON

        Returns:
            DataFrame con caracteristicas por fotografia
        """
        self.logger.info(f"Cargando caracteristicas de {len(rutas_json)} fotografias...")

        caracteristicas_list = []

        for ruta in rutas_json:
            try:
                caract = self.extraer_caracteristicas_json(ruta)
                caracteristicas_list.append(caract)
                self.logger.debug(f"  {caract['codigo_foto']}: OK")
            except Exception as e:
                self.logger.error(f"  Error en {ruta.name}: {e}")

        df_caracteristicas = pd.DataFrame(caracteristicas_list)

        self.logger.info(f"Caracteristicas extraidas: {len(df_caracteristicas)} fotos x {len(df_caracteristicas.columns)} columnas")

        return df_caracteristicas

    def fusionar_datos(self, df_metricas: pd.DataFrame,
                       df_caracteristicas: pd.DataFrame) -> pd.DataFrame:
        """
        Fusiona metricas de segmentacion con caracteristicas fotograficas.

        Args:
            df_metricas: DataFrame con metricas de Fase 2A
            df_caracteristicas: DataFrame con caracteristicas fotograficas

        Returns:
            DataFrame fusionado
        """
        self.logger.info("Fusionando metricas con caracteristicas fotograficas...")

        df_fusionado = df_metricas.merge(
            df_caracteristicas,
            on='codigo_foto',
            how='left'
        )

        # Verificar fotos sin match
        fotos_metricas = set(df_metricas['codigo_foto'].unique())
        fotos_caract = set(df_caracteristicas['codigo_foto'].unique())
        sin_match = fotos_metricas - fotos_caract

        if sin_match:
            self.logger.warning(f"Fotos sin caracteristicas: {len(sin_match)}")
            for foto in sin_match:
                self.logger.warning(f"  - {foto}")
        else:
            self.logger.info("Todas las fotos tienen caracteristicas asociadas")

        self.logger.info(f"Dataset fusionado: {len(df_fusionado)} filas x {len(df_fusionado.columns)} columnas")

        return df_fusionado

In [10]:
# =============================================================================
# CLASE: ANALIZADOR DE CORRELACIONES
# =============================================================================

class AnalizadorCorrelaciones:
    """
    Ejecuta analisis de correlaciones entre caracteristicas fotograficas
    y metricas de segmentacion.
    """

    def __init__(self, config: ConfiguracionFase2B, logger: logging.Logger):
        """
        Inicializa el analizador.

        Args:
            config: Configuracion de Fase 2B
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

    def calcular_correlaciones_globales(self, df: pd.DataFrame,
                                        metrica_objetivo: str = 'iou') -> pd.DataFrame:
        """
        Calcula correlaciones entre todas las caracteristicas fotograficas
        y una metrica objetivo.

        Args:
            df: DataFrame fusionado
            metrica_objetivo: Columna objetivo (default: IoU)

        Returns:
            DataFrame con correlaciones ordenadas por magnitud
        """
        self.logger.info(f"Calculando correlaciones globales vs {metrica_objetivo}...")

        # Identificar columnas de caracteristicas fotograficas
        cols_foto = [c for c in df.columns if c.startswith('foto_')]

        correlaciones = []

        for col in cols_foto:
            # Filtrar valores no nulos
            mask = df[col].notna() & df[metrica_objetivo].notna()
            n_obs = mask.sum()

            if n_obs >= self.config.min_observaciones_correlacion:
                try:
                    corr, pval = stats.pearsonr(
                        df.loc[mask, col],
                        df.loc[mask, metrica_objetivo]
                    )

                    correlaciones.append({
                        'caracteristica': col.replace('foto_', ''),
                        'correlacion': corr,
                        'p_valor': pval,
                        'n_observaciones': n_obs,
                        'significativo': pval < self.config.nivel_significancia
                    })
                except Exception as e:
                    self.logger.warning(f"Error calculando correlacion para {col}: {e}")

        df_corr = pd.DataFrame(correlaciones)

        if not df_corr.empty:
            # Ordenar por magnitud de correlacion
            df_corr = df_corr.sort_values('correlacion', key=abs, ascending=False)
            df_corr = df_corr.reset_index(drop=True)

        return df_corr

    def calcular_correlaciones_por_modelo(self, df: pd.DataFrame,
                                          metrica_objetivo: str = 'iou') -> pd.DataFrame:
        """
        Calcula correlaciones segmentadas por modelo.

        Args:
            df: DataFrame fusionado
            metrica_objetivo: Columna objetivo

        Returns:
            DataFrame con correlaciones por modelo
        """
        self.logger.info("Calculando correlaciones por modelo...")

        cols_foto = [c for c in df.columns if c.startswith('foto_')]
        modelos = df['modelo'].unique()

        resultados = []

        for modelo in modelos:
            df_modelo = df[df['modelo'] == modelo]
            n_total = len(df_modelo)

            for col in cols_foto:
                mask = df_modelo[col].notna() & df_modelo[metrica_objetivo].notna()
                n_obs = mask.sum()

                if n_obs >= self.config.min_observaciones_correlacion:
                    try:
                        corr, pval = stats.pearsonr(
                            df_modelo.loc[mask, col],
                            df_modelo.loc[mask, metrica_objetivo]
                        )

                        resultados.append({
                            'modelo': modelo,
                            'caracteristica': col.replace('foto_', ''),
                            'correlacion': corr,
                            'p_valor': pval,
                            'n_observaciones': n_obs,
                            'significativo': pval < self.config.nivel_significancia
                        })
                    except Exception:
                        pass

        df_corr_modelo = pd.DataFrame(resultados)

        return df_corr_modelo

    def identificar_top_predictores(self, df_corr_modelo: pd.DataFrame,
                                    top_n: int = 5) -> Dict[str, List[Dict]]:
        """
        Identifica los top-N predictores para cada modelo.

        Args:
            df_corr_modelo: DataFrame de correlaciones por modelo
            top_n: Numero de predictores a retornar

        Returns:
            Diccionario {modelo: [top predictores]}
        """
        resultado = {}

        for modelo in df_corr_modelo['modelo'].unique():
            df_m = df_corr_modelo[df_corr_modelo['modelo'] == modelo]
            df_m = df_m.sort_values('correlacion', key=abs, ascending=False)

            top = df_m.head(top_n).to_dict('records')
            resultado[modelo] = top

        return resultado

In [11]:

# =============================================================================
# CLASE: ANALIZADOR DE HIPOTESIS
# =============================================================================

class AnalizadorHipotesis:
    """
    Ejecuta tests estadisticos para hipotesis especificas sobre
    la relacion entre caracteristicas fotograficas y segmentacion.
    """

    def __init__(self, config: ConfiguracionFase2B, logger: logging.Logger):
        """
        Inicializa el analizador de hipotesis.

        Args:
            config: Configuracion de Fase 2B
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

    def test_hipotesis_apertura(self, df: pd.DataFrame) -> Dict:
        """
        H1: La apertura (bokeh) afecta la calidad de segmentacion.

        Divide fotografias en apertura abierta (<2.5) vs cerrada (>=2.5)
        y compara IoU medio.

        Args:
            df: DataFrame fusionado

        Returns:
            Diccionario con resultados del test
        """
        self.logger.info("Testing H1: Efecto de apertura (bokeh)...")

        mask = df['foto_apertura'].notna()
        df_h = df[mask].copy()

        umbral_apertura = 2.5
        df_h['grupo_apertura'] = df_h['foto_apertura'].apply(
            lambda x: 'abierta_bokeh' if x < umbral_apertura else 'cerrada_nitida'
        )

        grupo_abierta = df_h[df_h['grupo_apertura'] == 'abierta_bokeh']['iou']
        grupo_cerrada = df_h[df_h['grupo_apertura'] == 'cerrada_nitida']['iou']

        resultado = {
            'hipotesis': 'H1: Apertura afecta segmentacion',
            'variable': 'foto_apertura',
            'umbral': umbral_apertura,
            'grupo_1': {
                'nombre': 'abierta_bokeh (f < 2.5)',
                'n': len(grupo_abierta),
                'iou_mean': float(grupo_abierta.mean()),
                'iou_std': float(grupo_abierta.std())
            },
            'grupo_2': {
                'nombre': 'cerrada_nitida (f >= 2.5)',
                'n': len(grupo_cerrada),
                'iou_mean': float(grupo_cerrada.mean()),
                'iou_std': float(grupo_cerrada.std())
            }
        }

        # Test t de Student
        if len(grupo_abierta) > 1 and len(grupo_cerrada) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_abierta, grupo_cerrada)
            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }

            # Cohen's d
            cohens_d = self._calcular_cohens_d(grupo_abierta, grupo_cerrada)
            resultado['cohens_d'] = float(cohens_d)
            resultado['interpretacion_efecto'] = self._interpretar_cohens_d(cohens_d)

        return resultado

    def test_hipotesis_homogeneidad(self, df: pd.DataFrame) -> Dict:
        """
        H2: La homogeneidad del fondo afecta la segmentacion.

        Divide por mediana de homogeneidad GLCM.

        Args:
            df: DataFrame fusionado

        Returns:
            Diccionario con resultados del test
        """
        self.logger.info("Testing H2: Efecto de homogeneidad del fondo...")

        mask = df['foto_homogeneity'].notna()
        df_h = df[mask].copy()

        mediana = df_h['foto_homogeneity'].median()
        df_h['grupo_fondo'] = df_h['foto_homogeneity'].apply(
            lambda x: 'uniforme' if x > mediana else 'complejo'
        )

        grupo_uniforme = df_h[df_h['grupo_fondo'] == 'uniforme']['iou']
        grupo_complejo = df_h[df_h['grupo_fondo'] == 'complejo']['iou']

        resultado = {
            'hipotesis': 'H2: Homogeneidad fondo afecta segmentacion',
            'variable': 'foto_homogeneity',
            'umbral': float(mediana),
            'grupo_1': {
                'nombre': 'fondo_uniforme (> mediana)',
                'n': len(grupo_uniforme),
                'iou_mean': float(grupo_uniforme.mean()),
                'iou_std': float(grupo_uniforme.std())
            },
            'grupo_2': {
                'nombre': 'fondo_complejo (<= mediana)',
                'n': len(grupo_complejo),
                'iou_mean': float(grupo_complejo.mean()),
                'iou_std': float(grupo_complejo.std())
            }
        }

        if len(grupo_uniforme) > 1 and len(grupo_complejo) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_uniforme, grupo_complejo)
            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }

            cohens_d = self._calcular_cohens_d(grupo_uniforme, grupo_complejo)
            resultado['cohens_d'] = float(cohens_d)
            resultado['interpretacion_efecto'] = self._interpretar_cohens_d(cohens_d)

        return resultado

    def test_hipotesis_subexposicion(self, df: pd.DataFrame) -> Dict:
        """
        H3: La subexposicion (contraluz) afecta la segmentacion.

        Divide en contraluz severo (>10% subexpuesto) vs normal.

        Args:
            df: DataFrame fusionado

        Returns:
            Diccionario con resultados del test
        """
        self.logger.info("Testing H3: Efecto de subexposicion (contraluz)...")

        mask = df['foto_subexp_pct'].notna()
        df_h = df[mask].copy()

        umbral = 10.0
        df_h['grupo_exposicion'] = df_h['foto_subexp_pct'].apply(
            lambda x: 'contraluz_severo' if x > umbral else 'normal'
        )

        grupo_contraluz = df_h[df_h['grupo_exposicion'] == 'contraluz_severo']['iou']
        grupo_normal = df_h[df_h['grupo_exposicion'] == 'normal']['iou']

        resultado = {
            'hipotesis': 'H3: Subexposicion afecta segmentacion',
            'variable': 'foto_subexp_pct',
            'umbral': umbral,
            'grupo_1': {
                'nombre': 'contraluz_severo (> 10%)',
                'n': len(grupo_contraluz),
                'iou_mean': float(grupo_contraluz.mean()),
                'iou_std': float(grupo_contraluz.std())
            },
            'grupo_2': {
                'nombre': 'exposicion_normal (<= 10%)',
                'n': len(grupo_normal),
                'iou_mean': float(grupo_normal.mean()),
                'iou_std': float(grupo_normal.std())
            }
        }

        if len(grupo_contraluz) > 1 and len(grupo_normal) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_contraluz, grupo_normal)
            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }

            cohens_d = self._calcular_cohens_d(grupo_contraluz, grupo_normal)
            resultado['cohens_d'] = float(cohens_d)
            resultado['interpretacion_efecto'] = self._interpretar_cohens_d(cohens_d)

        return resultado

    def test_hipotesis_contraste(self, df: pd.DataFrame) -> Dict:
        """
        H4: El contraste de la imagen afecta la segmentacion.

        Divide por mediana de contraste RMS.

        Args:
            df: DataFrame fusionado

        Returns:
            Diccionario con resultados del test
        """
        self.logger.info("Testing H4: Efecto del contraste...")

        mask = df['foto_contraste_rms'].notna()
        df_h = df[mask].copy()

        mediana = df_h['foto_contraste_rms'].median()
        df_h['grupo_contraste'] = df_h['foto_contraste_rms'].apply(
            lambda x: 'alto' if x > mediana else 'bajo'
        )

        grupo_alto = df_h[df_h['grupo_contraste'] == 'alto']['iou']
        grupo_bajo = df_h[df_h['grupo_contraste'] == 'bajo']['iou']

        resultado = {
            'hipotesis': 'H4: Contraste afecta segmentacion',
            'variable': 'foto_contraste_rms',
            'umbral': float(mediana),
            'grupo_1': {
                'nombre': 'contraste_alto (> mediana)',
                'n': len(grupo_alto),
                'iou_mean': float(grupo_alto.mean()),
                'iou_std': float(grupo_alto.std())
            },
            'grupo_2': {
                'nombre': 'contraste_bajo (<= mediana)',
                'n': len(grupo_bajo),
                'iou_mean': float(grupo_bajo.mean()),
                'iou_std': float(grupo_bajo.std())
            }
        }

        if len(grupo_alto) > 1 and len(grupo_bajo) > 1:
            t_stat, p_val = stats.ttest_ind(grupo_alto, grupo_bajo)
            resultado['test_t'] = {
                'estadistico': float(t_stat),
                'p_valor': float(p_val),
                'significativo': p_val < self.config.nivel_significancia
            }

            cohens_d = self._calcular_cohens_d(grupo_alto, grupo_bajo)
            resultado['cohens_d'] = float(cohens_d)
            resultado['interpretacion_efecto'] = self._interpretar_cohens_d(cohens_d)

        return resultado

    def ejecutar_todos_los_tests(self, df: pd.DataFrame) -> Dict:
        """
        Ejecuta todas las hipotesis predefinidas.

        Args:
            df: DataFrame fusionado

        Returns:
            Diccionario con todos los resultados
        """
        return {
            'H1_apertura': self.test_hipotesis_apertura(df),
            'H2_homogeneidad': self.test_hipotesis_homogeneidad(df),
            'H3_subexposicion': self.test_hipotesis_subexposicion(df),
            'H4_contraste': self.test_hipotesis_contraste(df)
        }

    def _calcular_cohens_d(self, grupo1: pd.Series, grupo2: pd.Series) -> float:
        """
        Calcula el tamano de efecto Cohen's d.

        Args:
            grupo1: Primera muestra
            grupo2: Segunda muestra

        Returns:
            Valor de Cohen's d
        """
        n1, n2 = len(grupo1), len(grupo2)
        var1, var2 = grupo1.var(), grupo2.var()

        # Pooled standard deviation
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))

        if pooled_std == 0:
            return 0.0

        return (grupo1.mean() - grupo2.mean()) / pooled_std

    def _interpretar_cohens_d(self, d: float) -> str:
        """
        Interpreta el valor de Cohen's d segun convenciones.

        Args:
            d: Valor de Cohen's d

        Returns:
            Interpretacion textual
        """
        d_abs = abs(d)

        if d_abs < 0.2:
            return "insignificante"
        elif d_abs < 0.5:
            return "pequeno"
        elif d_abs < 0.8:
            return "mediano"
        else:
            return "grande"

In [12]:
# =============================================================================
# CLASE: ANALIZADOR DE TAMANO DE EFECTO ENTRE MODELOS
# =============================================================================

class AnalizadorEfectoModelos:
    """
    Calcula tamanos de efecto entre pares de modelos.
    """

    def __init__(self, config: ConfiguracionFase2B, logger: logging.Logger):
        """
        Inicializa el analizador.

        Args:
            config: Configuracion de Fase 2B
            logger: Logger para registro
        """
        self.config = config
        self.logger = logger

    def calcular_cohens_d_entre_modelos(self, df: pd.DataFrame,
                                        metrica: str = 'iou') -> pd.DataFrame:
        """
        Calcula Cohen's d para todos los pares de modelos.

        Args:
            df: DataFrame fusionado
            metrica: Metrica a comparar

        Returns:
            DataFrame con comparaciones pareadas
        """
        self.logger.info("Calculando Cohen's d entre modelos...")

        modelos = sorted(df['modelo'].unique())
        comparaciones = []

        for i, modelo1 in enumerate(modelos):
            for modelo2 in modelos[i+1:]:
                grupo1 = df[df['modelo'] == modelo1][metrica].dropna()
                grupo2 = df[df['modelo'] == modelo2][metrica].dropna()

                if len(grupo1) > 1 and len(grupo2) > 1:
                    d = self._cohens_d(grupo1, grupo2)
                    t_stat, p_val = stats.ttest_ind(grupo1, grupo2)

                    comparaciones.append({
                        'modelo_1': modelo1,
                        'modelo_2': modelo2,
                        'n_1': len(grupo1),
                        'n_2': len(grupo2),
                        'mean_1': float(grupo1.mean()),
                        'mean_2': float(grupo2.mean()),
                        'cohens_d': float(d),
                        'interpretacion': self._interpretar_d(d),
                        't_statistic': float(t_stat),
                        'p_valor': float(p_val),
                        'significativo': p_val < self.config.nivel_significancia
                    })

        return pd.DataFrame(comparaciones)

    def calcular_eta_cuadrado(self, df: pd.DataFrame,
                             metrica: str = 'iou') -> Dict:
        """
        Calcula eta cuadrado (ANOVA) para comparar todos los modelos.

        Args:
            df: DataFrame fusionado
            metrica: Metrica a comparar

        Returns:
            Diccionario con resultados de ANOVA
        """
        self.logger.info("Calculando eta cuadrado (ANOVA)...")

        # Preparar grupos por modelo
        grupos = [df[df['modelo'] == m][metrica].dropna().values
                  for m in df['modelo'].unique()]

        # ANOVA one-way
        f_stat, p_val = stats.f_oneway(*grupos)

        # Calcular eta cuadrado
        # SS_between / SS_total
        grand_mean = df[metrica].mean()
        ss_total = ((df[metrica] - grand_mean) ** 2).sum()

        ss_between = 0
        for modelo in df['modelo'].unique():
            grupo = df[df['modelo'] == modelo][metrica]
            ss_between += len(grupo) * (grupo.mean() - grand_mean) ** 2

        eta_squared = ss_between / ss_total if ss_total > 0 else 0

        return {
            'metrica': metrica,
            'n_modelos': len(df['modelo'].unique()),
            'n_total': len(df),
            'f_statistic': float(f_stat),
            'p_valor': float(p_val),
            'eta_squared': float(eta_squared),
            'interpretacion': self._interpretar_eta(eta_squared),
            'significativo': p_val < self.config.nivel_significancia
        }

    def _cohens_d(self, grupo1: pd.Series, grupo2: pd.Series) -> float:
        """Calcula Cohen's d entre dos grupos."""
        n1, n2 = len(grupo1), len(grupo2)
        var1, var2 = grupo1.var(), grupo2.var()
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))

        if pooled_std == 0:
            return 0.0
        return (grupo1.mean() - grupo2.mean()) / pooled_std

    def _interpretar_d(self, d: float) -> str:
        """Interpreta Cohen's d."""
        d_abs = abs(d)
        if d_abs < 0.2:
            return "insignificante"
        elif d_abs < 0.5:
            return "pequeno"
        elif d_abs < 0.8:
            return "mediano"
        return "grande"

    def _interpretar_eta(self, eta: float) -> str:
        """Interpreta eta cuadrado."""
        if eta < 0.01:
            return "insignificante"
        elif eta < 0.06:
            return "pequeno"
        elif eta < 0.14:
            return "mediano"
        return "grande"

In [13]:

# =============================================================================
# CLASE: GENERADOR DE ESTADISTICAS DESCRIPTIVAS
# =============================================================================

class GeneradorEstadisticas:
    """
    Genera estadisticas descriptivas del dataset fusionado.
    """

    def __init__(self, logger: logging.Logger):
        """
        Inicializa el generador.

        Args:
            logger: Logger para registro
        """
        self.logger = logger

    def estadisticas_por_modelo(self, df: pd.DataFrame) -> Dict:
        """
        Calcula estadisticas descriptivas por modelo.

        Args:
            df: DataFrame fusionado

        Returns:
            Diccionario con estadisticas por modelo
        """
        self.logger.info("Generando estadisticas por modelo...")

        stats_modelo = {}

        for modelo in df['modelo'].unique():
            df_m = df[df['modelo'] == modelo]

            stats_modelo[modelo] = {
                'n_evaluaciones': int(len(df_m)),
                'n_configuraciones': int(df_m['config_codigo'].nunique()),
                'n_fotos': int(df_m['codigo_foto'].nunique()),
                'iou': {
                    'mean': float(df_m['iou'].mean()),
                    'std': float(df_m['iou'].std()),
                    'min': float(df_m['iou'].min()),
                    'max': float(df_m['iou'].max()),
                    'median': float(df_m['iou'].median()),
                    'q25': float(df_m['iou'].quantile(0.25)),
                    'q75': float(df_m['iou'].quantile(0.75))
                },
                'dice': {
                    'mean': float(df_m['dice'].mean()) if 'dice' in df_m.columns else None,
                    'std': float(df_m['dice'].std()) if 'dice' in df_m.columns else None
                }
            }

        return stats_modelo

    def estadisticas_por_foto(self, df: pd.DataFrame) -> Dict:
        """
        Calcula estadisticas descriptivas por fotografia.

        Args:
            df: DataFrame fusionado

        Returns:
            Diccionario con estadisticas por foto
        """
        self.logger.info("Generando estadisticas por fotografia...")

        stats_foto = {}

        for foto in df['codigo_foto'].unique():
            df_f = df[df['codigo_foto'] == foto]

            stats_foto[foto] = {
                'n_evaluaciones': int(len(df_f)),
                'iou_mean': float(df_f['iou'].mean()),
                'iou_std': float(df_f['iou'].std()),
                'mejor_modelo': df_f.loc[df_f['iou'].idxmax(), 'modelo'],
                'mejor_config': df_f.loc[df_f['iou'].idxmax(), 'config_codigo'],
                'mejor_iou': float(df_f['iou'].max())
            }

        return stats_foto

    def ranking_global(self, df: pd.DataFrame, top_n: int = 20) -> pd.DataFrame:
        """
        Genera ranking global de configuraciones por IoU.

        Args:
            df: DataFrame fusionado
            top_n: Numero de configuraciones a incluir

        Returns:
            DataFrame con ranking
        """
        self.logger.info(f"Generando ranking TOP-{top_n}...")

        ranking = df.groupby(['modelo', 'config_codigo']).agg({
            'iou': ['mean', 'std', 'min', 'max', 'count']
        }).round(4)

        ranking.columns = ['iou_mean', 'iou_std', 'iou_min', 'iou_max', 'n_fotos']
        ranking = ranking.reset_index()
        ranking = ranking.sort_values('iou_mean', ascending=False)

        return ranking.head(top_n)

In [14]:

# =============================================================================
# CLASE: ORQUESTADOR FASE 2B
# =============================================================================

class OrquestadorFase2B:
    """
    Orquesta la ejecucion completa de Fase 2B.
    """

    def __init__(self, config: ConfiguracionFase2B):
        """
        Inicializa el orquestador.

        Args:
            config: Configuracion de Fase 2B
        """
        self.config = config
        self.logger = self._configurar_logger()

        # Componentes
        self.cargador = CargadorDatosFase2B(config, self.logger)
        self.analizador_corr = AnalizadorCorrelaciones(config, self.logger)
        self.analizador_hip = AnalizadorHipotesis(config, self.logger)
        self.analizador_efecto = AnalizadorEfectoModelos(config, self.logger)
        self.generador_stats = GeneradorEstadisticas(self.logger)

        # Resultados
        self.df_fusionado = None
        self.df_caracteristicas = None
        self.correlaciones_globales = None
        self.correlaciones_por_modelo = None
        self.resultados_hipotesis = None
        self.tamano_efecto_modelos = None
        self.estadisticas = None

    def _configurar_logger(self) -> logging.Logger:
        """Configura el logger para Fase 2B."""
        logger = logging.getLogger('Fase2B')
        logger.setLevel(logging.INFO)

        if not logger.handlers:
            handler = logging.StreamHandler(sys.stdout)
            handler.setLevel(logging.INFO)
            formatter = logging.Formatter(
                '[%(asctime)s] %(levelname)-8s | %(message)s',
                datefmt='%H:%M:%S'
            )
            handler.setFormatter(formatter)
            logger.addHandler(handler)

        return logger

    def ejecutar(self, rutas_csv: Dict[str, Path],
                 rutas_json: List[Path]) -> None:
        """
        Ejecuta el pipeline completo de Fase 2B.

        Args:
            rutas_csv: Diccionario {modelo: ruta_csv}
            rutas_json: Lista de rutas a JSONs de caracteristicas
        """
        self.logger.info("=" * 70)
        self.logger.info("FASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES")
        self.logger.info("=" * 70)

        inicio = datetime.now()

        # Paso 1: Cargar y consolidar datos
        self.logger.info("\n[PASO 1/7] Cargando y consolidando datos...")
        df_metricas = self.cargador.cargar_csvs_metricas(rutas_csv)
        self.df_caracteristicas = self.cargador.cargar_caracteristicas_fotograficas(rutas_json)
        self.df_fusionado = self.cargador.fusionar_datos(df_metricas, self.df_caracteristicas)

        # Paso 2: Correlaciones globales
        self.logger.info("\n[PASO 2/7] Calculando correlaciones globales...")
        self.correlaciones_globales = self.analizador_corr.calcular_correlaciones_globales(
            self.df_fusionado
        )

        # Paso 3: Correlaciones por modelo
        self.logger.info("\n[PASO 3/7] Calculando correlaciones por modelo...")
        self.correlaciones_por_modelo = self.analizador_corr.calcular_correlaciones_por_modelo(
            self.df_fusionado
        )

        # Paso 4: Tests de hipotesis
        self.logger.info("\n[PASO 4/7] Ejecutando tests de hipotesis...")
        self.resultados_hipotesis = self.analizador_hip.ejecutar_todos_los_tests(
            self.df_fusionado
        )

        # Paso 5: Tamano de efecto entre modelos
        self.logger.info("\n[PASO 5/7] Calculando tamano de efecto entre modelos...")
        self.tamano_efecto_modelos = {
            'cohens_d_pareado': self.analizador_efecto.calcular_cohens_d_entre_modelos(
                self.df_fusionado
            ).to_dict('records'),
            'anova_eta_squared': self.analizador_efecto.calcular_eta_cuadrado(
                self.df_fusionado
            )
        }

        # Paso 6: Estadisticas descriptivas
        self.logger.info("\n[PASO 6/7] Generando estadisticas descriptivas...")
        self.estadisticas = {
            'por_modelo': self.generador_stats.estadisticas_por_modelo(self.df_fusionado),
            'por_foto': self.generador_stats.estadisticas_por_foto(self.df_fusionado),
            'ranking_top20': self.generador_stats.ranking_global(self.df_fusionado, 20).to_dict('records')
        }

        # Paso 7: Guardar resultados
        self.logger.info("\n[PASO 7/7] Guardando resultados...")
        self._guardar_resultados()

        duracion = (datetime.now() - inicio).total_seconds()

        self.logger.info("\n" + "=" * 70)
        self.logger.info("FASE 2B COMPLETADA")
        self.logger.info("=" * 70)
        self.logger.info(f"Duracion: {duracion:.1f} segundos")
        self.logger.info(f"Evaluaciones procesadas: {len(self.df_fusionado)}")
        self.logger.info(f"Modelos: {self.df_fusionado['modelo'].nunique()}")
        self.logger.info(f"Fotografias: {self.df_fusionado['codigo_foto'].nunique()}")

    def _guardar_resultados(self) -> None:
        """Guarda todos los resultados en archivos."""
        # Crear directorio de salida
        self.config.ruta_salida.mkdir(parents=True, exist_ok=True)

        # CSV fusionado
        ruta_fusionado = self.config.ruta_salida / 'metricas_fusionadas.csv'
        self.df_fusionado.to_csv(ruta_fusionado, index=False)
        self.logger.info(f"  Guardado: {ruta_fusionado}")

        # Caracteristicas fotograficas
        ruta_caract = self.config.ruta_salida / 'caracteristicas_fotograficas.csv'
        self.df_caracteristicas.to_csv(ruta_caract, index=False)
        self.logger.info(f"  Guardado: {ruta_caract}")

        # Correlaciones globales
        ruta_corr_global = self.config.ruta_salida / 'correlaciones_globales.csv'
        self.correlaciones_globales.to_csv(ruta_corr_global, index=False)
        self.logger.info(f"  Guardado: {ruta_corr_global}")

        # Correlaciones por modelo
        ruta_corr_modelo = self.config.ruta_salida / 'correlaciones_por_modelo.csv'
        self.correlaciones_por_modelo.to_csv(ruta_corr_modelo, index=False)
        self.logger.info(f"  Guardado: {ruta_corr_modelo}")

        # Test de hipotesis
        ruta_hipotesis = self.config.ruta_salida / 'test_hipotesis.json'
        with open(ruta_hipotesis, 'w', encoding='utf-8') as f:
            json.dump(convertir_a_serializable(self.resultados_hipotesis), f, indent=2, ensure_ascii=False)
        self.logger.info(f"  Guardado: {ruta_hipotesis}")

        # Tamano de efecto
        ruta_efecto = self.config.ruta_salida / 'tamano_efecto.json'
        with open(ruta_efecto, 'w', encoding='utf-8') as f:
            json.dump(convertir_a_serializable(self.tamano_efecto_modelos), f, indent=2, ensure_ascii=False)
        self.logger.info(f"  Guardado: {ruta_efecto}")

        # Estadisticas
        ruta_stats = self.config.ruta_salida / 'estadisticas_descriptivas.json'
        with open(ruta_stats, 'w', encoding='utf-8') as f:
            json.dump(convertir_a_serializable(self.estadisticas), f, indent=2, ensure_ascii=False)
        self.logger.info(f"  Guardado: {ruta_stats}")

    def imprimir_resumen(self) -> None:
        """Imprime un resumen de los resultados principales."""
        print("\n" + "=" * 70)
        print("RESUMEN DE RESULTADOS FASE 2B")
        print("=" * 70)

        # Correlaciones globales significativas
        print("\nCORRELACIONES GLOBALES SIGNIFICATIVAS (p < 0.05):")
        print("-" * 50)
        corr_sig = self.correlaciones_globales[
            self.correlaciones_globales['significativo']
        ]
        for _, row in corr_sig.head(10).iterrows():
            direccion = "+" if row['correlacion'] > 0 else "-"
            print(f"  {row['caracteristica']:<25} r={row['correlacion']:>7.4f} ({direccion})")

        # IoU por modelo
        print("\nRENDIMIENTO POR MODELO (IoU):")
        print("-" * 50)
        for modelo, stats in sorted(
            self.estadisticas['por_modelo'].items(),
            key=lambda x: x[1]['iou']['mean'],
            reverse=True
        ):
            iou = stats['iou']
            print(f"  {modelo:<15} mean={iou['mean']:.4f} std={iou['std']:.4f} "
                  f"[{iou['min']:.4f}, {iou['max']:.4f}]")

        # Hipotesis
        print("\nRESULTADOS DE HIPOTESIS:")
        print("-" * 50)
        for key, result in self.resultados_hipotesis.items():
            sig = "SI" if result.get('test_t', {}).get('significativo', False) else "NO"
            efecto = result.get('interpretacion_efecto', 'N/A')
            print(f"  {key}: Significativo={sig}, Efecto={efecto}")

        # ANOVA
        anova = self.tamano_efecto_modelos['anova_eta_squared']
        print(f"\nANOVA GLOBAL:")
        print(f"  F={anova['f_statistic']:.2f}, p={anova['p_valor']:.2e}")
        print(f"  Eta^2={anova['eta_squared']:.4f} ({anova['interpretacion']})")

In [15]:
# =============================================================================
# FUNCION PRINCIPAL
# =============================================================================

def ejecutar_fase2b(ruta_base_tfm: str,
                    rutas_csv: Dict[str, str],
                    rutas_json: List[str]) -> OrquestadorFase2B:
    """
    Ejecuta el pipeline completo de Fase 2B.

    Args:
        ruta_base_tfm: Ruta base del proyecto TFM
        rutas_csv: Diccionario {modelo: ruta_csv}
        rutas_json: Lista de rutas a JSONs de caracteristicas

    Returns:
        OrquestadorFase2B con resultados
    """
    config = ConfiguracionFase2B(
        ruta_base_tfm=Path(ruta_base_tfm)
    )

    orquestador = OrquestadorFase2B(config)

    # Convertir rutas a Path
    rutas_csv_path = {k: Path(v) for k, v in rutas_csv.items()}
    rutas_json_path = [Path(r) for r in rutas_json]

    orquestador.ejecutar(rutas_csv_path, rutas_json_path)
    orquestador.imprimir_resumen()

    return orquestador

In [17]:
# =============================================================================
# PUNTO DE ENTRADA PARA PRUEBAS LOCALES
# =============================================================================

if __name__ == "__main__":

    from google.colab import drive
    drive.mount('/content/drive')

    # ===========================================================================
    # CONFIGURACIÓN DE RUTAS
    # ===========================================================================

    RUTA_BASE_TFM = Path("/content/drive/MyDrive/TFM")
    RUTA_FASE2A = RUTA_BASE_TFM / "3_Analisis" / "fase2_evaluacion" / "metricas_agregadas"
    RUTA_CARACTERISTICAS = RUTA_BASE_TFM / "1_Caracteristicas" / "json"

    # CSVs de métricas por modelo
    rutas_csv = {
        'yolov8': RUTA_FASE2A / "todas_metricas.csv",
        'bodypix': RUTA_FASE2A / "bodypix_todas_metricas.csv",
        'mask2former': RUTA_FASE2A / "m2f_todas_metricas.csv",
        'oneformer': RUTA_FASE2A / "of_todas_metricas.csv",
        'sam2': RUTA_FASE2A / "s2_todas_metricas.csv",
        'sam2_prompts': RUTA_FASE2A / "s2p_todas_metricas.csv"
    }

    # JSONs de características fotográficas
    rutas_json = sorted(RUTA_CARACTERISTICAS.glob("_DSC*_caracteristicas.json"))

    # Verificar archivos
    print("Verificando archivos...")
    for modelo, ruta in rutas_csv.items():
        existe = "OK" if ruta.exists() else "NO ENCONTRADO"
        print(f"  {modelo}: {existe}")
    print(f"  JSONs encontrados: {len(rutas_json)}")

    # Ejecutar Fase 2B
    orquestador = ejecutar_fase2b(
        ruta_base_tfm=str(RUTA_BASE_TFM),
        rutas_csv={k: str(v) for k, v in rutas_csv.items()},
        rutas_json=[str(r) for r in rutas_json]
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Verificando archivos...
  yolov8: OK
  bodypix: OK
  mask2former: OK
  oneformer: OK
  sam2: OK
  sam2_prompts: OK
  JSONs encontrados: 20
[21:44:14] INFO     | ======================================================================


INFO:Fase2B:======================================================================


[21:44:14] INFO     | FASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES


INFO:Fase2B:FASE 2B - FUSION DE DATOS Y ANALISIS DE CORRELACIONES


[21:44:14] INFO     | ======================================================================


...

[21:44:22] INFO     | Modelos: 6


INFO:Fase2B:Modelos: 6


[21:44:22] INFO     | Fotografias: 20


INFO:Fase2B:Fotografias: 20



RESUMEN DE RESULTADOS FASE 2B

CORRELACIONES GLOBALES SIGNIFICATIVAS (p < 0.05):
--------------------------------------------------
  homogeneity               r=-0.1407 (-)
  haralick_contrast         r= 0.1352 (+)
  haralick_entropy          r= 0.1302 (+)
  nitidez_laplacian         r= 0.1235 (+)
  snr                       r=-0.1129 (-)
  exposicion                r=-0.0929 (-)
  freq_ratio_alta           r= 0.0760 (+)
  apertura                  r= 0.0747 (+)
  entropia_color            r= 0.0682 (+)
  saturacion                r= 0.0638 (+)

RENDIMIENTO POR MODELO (IoU):
--------------------------------------------------
  yolov8          mean=0.9498 std=0.0527 [0.0010, 0.9803]
  oneformer       mean=0.8790 std=0.2185 [0.1692, 0.9908]
  mask2former     mean=0.6933 std=0.3572 [0.0000, 0.9878]
  bodypix         mean=0.6559 std=0.1740 [0.2022, 0.9535]
  sam2_prompts    mean=0.4614 std=0.3746 [0.0000, 0.9861]
  sam2            mean=0.3077 std=0.3616 [0.0000, 0.9812]

RESULTADOS DE HI